# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770865532786                   -0.52    9.0         
  2   -2.772148341850       -2.89       -1.32    1.0    105ms
  3   -2.772170282440       -4.66       -2.44    1.0    115ms
  4   -2.772170650894       -6.43       -3.15    1.0    127ms
  5   -2.772170722876       -7.14       -4.39    2.0    124ms
  6   -2.772170722945      -10.16       -4.53    1.0    136ms
  7   -2.772170723012      -10.17       -5.25    1.0    114ms
  8   -2.772170723014      -11.86       -5.41    1.0    116ms
  9   -2.772170723015      -11.92       -6.31    1.0    134ms
 10   -2.772170723015      -13.11       -6.57    2.0    133ms
 11   -2.772170723015      -14.88       -6.92    1.0    133ms
 12   -2.772170723015      -14.15       -7.64    1.0    120ms
 13   -2.772170723015   +  -14.07       -7.92    2.0    154ms
 14   -2.772170723015      -14.21       -9.03    1.0    134ms
n     Ene

1.7735580741994266

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770681892975                   -0.53    8.0         
  2   -2.772050048792       -2.86       -1.30    1.0    105ms
  3   -2.772082459433       -4.49       -2.63    1.0    111ms
  4   -2.772083415250       -6.02       -4.00    2.0    160ms
  5   -2.772083417713       -8.61       -4.56    2.0    125ms
  6   -2.772083417806      -10.03       -5.39    1.0    139ms
  7   -2.772083417810      -11.33       -5.97    2.0    128ms
  8   -2.772083417811      -13.03       -6.94    1.0    116ms
  9   -2.772083417811      -13.57       -7.57    2.0    151ms
 10   -2.772083417811   +  -14.27       -8.25    1.0    126ms

Polarizability via ForwardDiff:       1.772534957566956
Polarizability via finite difference: 1.7735580741994266
